<a href="https://colab.research.google.com/github/gym-guidance/api/blob/main/gym_guidance_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install flask-ngrok requests
!pip install pyngrok

In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler
import requests
from io import BytesIO
from pyngrok import ngrok  # Import pyngrok

from pyngrok import ngrok

# Replace 'your-authtoken' with the token you got from the ngrok dashboard
ngrok.set_auth_token("2qPmpLlE2YB3gPtuCMA3J4nQQYq_6KRyYCFf7Vk6opsCwAV1w")

# Load scaler
scalerurl = 'https://raw.githubusercontent.com/gym-guidance/api/main/scaler.pkl'

# URL of the raw .pkl file
url = "https://raw.githubusercontent.com/gym-guidance/api/main/gym_prediction_model.pkl"

# Fetch the file from GitHub
response = requests.get(url)
response1 = requests.get(scalerurl)

# Load model and scaler
model = joblib.load(BytesIO(response.content))
scaler = joblib.load(BytesIO(response1.content))

# Initialize Flask app
app = Flask(__name__)

# Start ngrok tunnel
public_url = ngrok.connect(5000)  # Start a public ngrok tunnel on port 5000
print(f" * ngrok tunnel \"{public_url}\" -> http://127.0.0.1:5000")

@app.route('/test', methods=['GET'])
def test():
  return "Hello World";

# API endpoint for exercise recommendation
@app.route('/recommendexercise', methods=['GET'])
def recommend_exercise():
    data = request.get_json()
    age = data['age']
    weight = data['weight']
    height = data['height']
    gender = data['gender']
    bmi = data['bmi']

    # Convert gender to a numerical value
    gender_numeric = 0 if gender == 'male' else 1  # Assuming only 'male' or 'female'

    # Preprocess the input data
    user_data = np.array([[age, weight, height, gender_numeric, bmi]])

    # Scale the input data
    user_data_scaled = scaler.transform(user_data)

    # Get the prediction
    recommendation = model.predict(user_data_scaled)

    return jsonify({"recommended_exercise": recommendation[0]})

if __name__ == '__main__':
    app.run()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


 * ngrok tunnel "NgrokTunnel: "https://ba87-34-125-22-116.ngrok-free.app" -> "http://localhost:5000"" -> http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
